In [ ]:
import sys
sys.version

In [ ]:
import pydicom

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rai
import raicontours

from raicontours import TG263

In [ ]:
raicontours.cfg

In [ ]:
rai_model = rai.load_model(cfg=raicontours.cfg)

In [ ]:
image_paths, structure_path = rai.download_deepmind_example("rai-data")
structure_path

In [ ]:
x_grid, y_grid, image_stack, image_uids = rai.paths_to_image_stack_hfs(
    image_paths
)

In [ ]:
z = [35, 45, 55]
y = [140, 155, 175, 195, 215, 230]
x = [210, 230, 250, 270, 290, 310]

predicted_masks = rai.inference_over_jittered_grid(
    model=rai_model, image_stack=image_stack, grid=(z, y, x)
)

In [ ]:
predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    x_grid, y_grid, predicted_masks
)

predicted_contours_by_structure.keys()

In [ ]:
rai.plot_contours_by_structure(
    x_grid, y_grid, image_stack, predicted_contours_by_structure
)

## Compare to DICOM-RT Structure

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
align_map = {
    "Orbit-Lt": [TG263.Eye_L],
    "Orbit-Rt": [TG263.Eye_R],
    "Lacrimal-Lt": [TG263.Glnd_Lacrimal_L],
    "Lacrimal-Rt": [TG263.Glnd_Lacrimal_R],
    "Lens-Lt": [TG263.Lens_L],
    "Lens-Rt": [TG263.Lens_R],
    "Optic-Nerve-Lt": [TG263.OpticNrv_L],
    "Optic-Nerve-Rt": [TG263.OpticNrv_R],
}

In [ ]:
structure_names = list(align_map.keys())
structure_names

In [ ]:
dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
    ds=structure_ds, image_uids=image_uids, structure_names=structure_names
)
dicom_contours_by_structure.keys()

In [ ]:
aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
    predicted_contours_by_structure, align_map
)
aligned_predicted_contours_by_structure.keys()

In [ ]:
dice = {}
for name in align_map:
    dice[name] = rai.dice_from_contours_by_slice(
        dicom_contours_by_structure[name],
        aligned_predicted_contours_by_structure[name],
    )

dice

In [ ]:
combined_contours_by_structure = {
    **predicted_contours_by_structure,
    **dicom_contours_by_structure,
}

rai.plot_contours_by_structure(
    x_grid, y_grid, image_stack, combined_contours_by_structure, align_map
)